In [1]:

import time
import requests
from requests.exceptions import Timeout, ConnectionError, HTTPError

import unicodedata
import re

from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import json

from diccionario_ligas import ligas_dict

DEBUG=False

request_headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

url_base = 'https://www.transfermarkt.es'


lista_comunitarios = ['Alemania', 'Austria', 'Bélgica', 'Bulgaria', 'Chipre',
                      'Croacia' 'Dinamarca', 'Eslovaquia' 'Eslovenia',
                      'España', 'Estonia', 'Finlandia', 'Francia', 'Grecia',
                      'Hungría', 'Irlanda', 'Islandia', 'Italia', 'Letonia',
                      'Liechtenstein', 'Lituania', 'Luxemburgo', 'Malta',
                      'Noruega', 'Países Bajos', 'Polonia', 'Portugal',
                      'República Checa', 'Rumania', 'Suecia', 'Suiza']
# https://www.goal.com/es-ar/noticias/que-es-futbolista-comunitario-extracomunitario/epa2dp4zli5z1rtclyu9lmgut


In [2]:

def parse_name(name):
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
    parsed_name = '-'.join(name.lower().split())
    return parsed_name


def parse_club_name(name):
    # Convertir a minúsculas y normalizar eliminando acentos
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

    # Eliminar símbolos no deseados (mantener solo letras, números y espacios)
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)

    # Convertir a formato con guiones
    parsed_name = '-'.join(name.lower().split())

    return parsed_name
    
def normalize_string(s):
    """Normaliza una cadena de texto eliminando acentos y convirtiendo a minúsculas"""
    if isinstance(s, str):
        s = s.strip()
        s = s.lower()
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    return s

def normalize_age(age):
    # Verificar si el valor es un string en formato 'años-días'
    if isinstance(age, str) and '-' in age:
        years, _ = age.split('-')  # Ignorar la parte de días
        return int(years) if years.strip().isdigit() else None  # Convertir a entero solo los años
    return int(age) if age.strip().isdigit() else None  # Si ya es entero, devolverlo tal cual
    

def make_request(url, params=None, retries=5, timeout=20, retry_delay=5, **kwargs):
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=timeout, **kwargs)
            response.raise_for_status()  # Verifica si hubo un error en la respuesta
            
            # Si la respuesta es exitosa, devolverla
            return response
        except Timeout:
            print(f"Intento {attempt + 1} de {retries}: Tiempo de espera excedido. Reintentando...")
        except ConnectionError as e:
            print(f"Intento {attempt + 1} de {retries}: Error de conexión: {e}. Reintentando...")
        except HTTPError as e:
            # Manejar los códigos 502, 504 y 429
            if response.status_code in [502, 504]:
                error_type = "502 Bad Gateway" if response.status_code == 502 else "504 Gateway Time-out"
                print(f"Intento {attempt + 1} de {retries}: Error {error_type}. Reintentando en {retry_delay} segundos...")
                time.sleep(retry_delay)
            elif response.status_code == 429:
                print(f"Intento {attempt + 1} de {retries}: Error 429 Too Many Requests. Aplicando espera exponencial.")
                retry_delay *= 2  # Aumenta el tiempo de espera exponencialmente
                time.sleep(retry_delay)
            else:
                print(f"Error HTTP: {e}. No es un 502, 504, ni 429, deteniendo reintentos.")
                return None
        # Agregar un breve retraso antes de reintentar
        time.sleep(retry_delay)
    
    print("Error: No se pudo establecer la conexión después de varios intentos.")
    return None

In [3]:
def scrape_transfermarkt(club_id, club_name, season):
    club_name_parsed = parse_club_name(club_name)
    url = f'https://www.transfermarkt.es/{club_name_parsed}/kader/verein/{club_id}/plus/0/galerie/0?saison_id={str(season-1)}'
    
    # Obtener el HTML de la página
    response = make_request(url, headers=request_headers)
    if response is None:
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Buscar la tabla correcta en el HTML
    tablas = soup.find_all('table')
    tbody = tablas[1].find('tbody')  # Usamos la segunda tabla

    jugadores = []

    for fila in tbody.find_all('tr', recursive=False):
        # ID y Nombre del jugador
        jugador_link = fila.find('a', href=re.compile(r'/profil/spieler/'))
        
        if jugador_link:
            jugador_id = jugador_link['href'].split('/')[-1]  # Extraer el ID de la URL
            nombre = normalize_string(jugador_link.text.strip())
        else:
            continue  # Saltar si no encontramos un enlace de perfil del jugador
        
        # Posición principal (ej: "Portero", "Defensa") usando regex para la clase
        posicion_cell = fila.find('td', class_=re.compile(r'^zentriert rueckennummer bg_'))
        if posicion_cell:
            posicion = posicion_cell.get('title', 'Desconocido').split(' ')[0]  # Ejemplo: "Portero", "Defensa"
        else:
            posicion = 'Desconocido'

        # Edad del jugador
        try:
            edad_cell = fila.find_all('td', class_='zentriert')[1]
            edad = normalize_age(edad_cell.text.strip()) 
        except (IndexError, AttributeError):
            edad = '-'

        # Agregar el jugador a la lista
        jugadores.append({
            'id': jugador_id,
            'nombre': nombre,
            'posicion': posicion,
            'edad': edad
        })

    return jugadores


In [4]:
def scrape_fbref(club_id, season, DEBUG=True):
    url = f'https://fbref.com/en/squads/{club_id}/{season}/'
    response = make_request(url)
    if response is None:
        season_eur = str(season-1) + '-' + str(season)
        if DEBUG:
            print(f'reintentando url con season: {season_eur}')
        url = f'https://fbref.com/en/squads/{club_id}/str{season_eur}/'
        response = make_request(url)
        
        if response is None:
            return {}
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Seleccionamos la primera tabla (que contiene la información de jugadores)
    tablas = soup.find_all('table')
    if not tablas:
        print("No se encontró la tabla de jugadores.")
        return []

    tbody = tablas[0].find('tbody')
    jugadores = []
    
    for row in tbody.find_all('tr'):
        # Extraer ID del jugador
        jugador_celda = row.find('th', {'data-stat': 'player'})
        try:
            jugador_id = jugador_celda['data-append-csv'] if jugador_celda else None
        except KeyError:
            continue
        jugador_nombre = normalize_string(jugador_celda.text.strip()) if jugador_celda else 'Desconocido'
        
        # Extraer posición
        posicion_celda = row.find('td', {'data-stat': 'position'})
        posicion = posicion_celda.text.strip() if posicion_celda else 'N/A'
        
        # Extraer edad
        edad_celda = row.find('td', {'data-stat': 'age'})
        if edad_celda:
            edad = normalize_age(edad_celda.text.strip()) 
        else:
            edad = '-'

        # Agregar información del jugador al diccionario
        jugadores.append({
            'id': jugador_id,
            'nombre': jugador_nombre,
            'posicion': posicion,
            'edad': edad
        })

    return jugadores


In [5]:
from fuzzywuzzy import fuzz, process


# Diccionario de mapeo de posiciones de Transfermarkt a FBRef
position_mapping = {
    'Portero': 'GK',
    'Defensa': 'DF',
    'Medio': 'MF',
    'Delantero': 'FW'
}

# Cargar el diccionario de mapeo de clubes
with open('club_mapping_with_code.json', 'r') as f:
    club_mapping = json.load(f)

# Función para normalizar posiciones usando el mapeo solo en Transfermarkt
def normalizar_posicion(posiciones, mapping):
    """
    Convierte una cadena de posiciones de Transfermarkt al formato de FBRef.
    Si ya está en el formato de FBRef, lo mantiene igual.
    """
    posiciones_normalizadas = set()
    for pos in posiciones.split(','):
        pos = pos.strip()
        # Traducir posiciones de Transfermarkt
        posiciones_normalizadas.add(mapping.get(pos, pos))
    return posiciones_normalizadas


# Función para emparejar jugadores entre Transfermarkt y FBRef usando fuzzy matching y posición
def match_players(transfer_players, fbref_players, threshold=80):
    matched_players = {}
    unmatched_players = []

    for t_player in transfer_players:
        # Normalizamos la posición del jugador en Transfermarkt
        pos_transfer = normalizar_posicion(t_player['posicion'], position_mapping)
        
        # Filtrar jugadores de FBRef con posiciones compatibles
        fbref_candidates = [
            p for p in fbref_players
            if pos_transfer & normalizar_posicion(p['posicion'], position_mapping)
        ]

        if fbref_candidates:
            # Encuentra el mejor candidato usando fuzzy matching
            best_match = process.extractOne(
                t_player['nombre'],
                [p['nombre'] for p in fbref_candidates],
                scorer=fuzz.token_set_ratio
            )

            if best_match:
                fbref_player = next(p for p in fbref_candidates if p['nombre'] == best_match[0])

                try:
                    check_edad = abs(t_player['edad'] - fbref_player['edad']) <= 2
                except TypeError:
                    check_edad = True
                
                # Verificar diferencia de edad
                if best_match[1] >= threshold and check_edad:
                    matched_players[t_player['id']] = {
                        'id_transfermarkt': t_player['id'],
                        'nombre_transfermarkt': t_player['nombre'],
                        'posicion_transfermarkt': t_player['posicion'],
                        'id_fbref': fbref_player['id'],
                        'nombre_fbref': fbref_player['nombre'],
                        'posicion_fbref': fbref_player['posicion'],
                        'match_score': best_match[1]
                    }
                else:
                    unmatched_players.append({
                        'id_transfermarkt': t_player['id'],
                        'nombre_transfermarkt': t_player['nombre'],
                        'posicion_transfermarkt': t_player['posicion'],
                        'id_fbref': fbref_player['id'],
                        'mejor_nombre_fbref': fbref_player['nombre'],
                        'mejor_score': best_match[1]
                    })
            else:
                unmatched_players.append({
                    'id_transfermarkt': t_player['id'],
                    'nombre_transfermarkt': t_player['nombre'],
                    'posicion_transfermarkt': t_player['posicion'],
                    'mejor_nombre_fbref': None,
                    'mejor_score': None
                })

    return matched_players, unmatched_players


In [6]:
import json
import pandas as pd
import time
import logging
from fuzzywuzzy import fuzz, process
import os

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler("scraping.log"), logging.StreamHandler()]
)

# Archivos de almacenamiento
PLAYER_MAPPING_FILE = 'player_id_mapping.json'
UNMATCHED_PLAYERS_FILE = 'unmatched_players.csv'

# Función para cargar combinaciones ya procesadas desde 'player_id_mapping.json'
def load_scraped_combinations():
    if os.path.exists(PLAYER_MAPPING_FILE):
        with open(PLAYER_MAPPING_FILE, 'r') as f:
            data = json.load(f)
            return set(tuple(combo) for combo in data.get('scraped_combinations', [])), data.get('players', {})
    return set(), {}

# Función para guardar combinaciones y jugadores procesados en un archivo JSON
def save_scraped_data(scraped_combinations, player_data):
    with open(PLAYER_MAPPING_FILE, 'w') as f:
        json.dump({
            'scraped_combinations': list(scraped_combinations),
            'players': player_data
        }, f, indent=4)

# Cargar combinaciones y datos de jugadores procesados
scraped_combinations, all_matches = load_scraped_combinations()

# Guardado parcial de resultados y combinaciones procesadas
def save_partial_results(matches, unmatched, scraped_combinations):
    # Guardar coincidencias y combinaciones procesadas
    save_scraped_data(scraped_combinations, matches)
    
    # Guardar jugadores no coincidentes en un archivo CSV, manteniendo previos
    if os.path.exists(UNMATCHED_PLAYERS_FILE):
        existing_unmatched = pd.read_csv(UNMATCHED_PLAYERS_FILE)
        unmatched_df = pd.concat([existing_unmatched, pd.DataFrame(unmatched)], ignore_index=True)
    else:
        unmatched_df = pd.DataFrame(unmatched)
    unmatched_df.to_csv(UNMATCHED_PLAYERS_FILE, index=False)

# Función principal para generar el diccionario de jugadores
def save_unmatched_to_csv(file_path, unmatched, matched_players):
    """
    Guarda los resultados no emparejados en un archivo CSV, garantizando la estructura
    y excluyendo jugadores ya emparejados.
    """
    if not unmatched:
        logging.info("No hay jugadores no emparejados para guardar.")
        return

    # Asegurarse de que los `unmatched` tengan las claves requeridas
    required_columns = ['id_transfermarkt', 'nombre_transfermarkt', 'posicion_transfermarkt', 
                        'id_fbref', 'mejor_nombre_fbref', 'mejor_score']
    
    for entry in unmatched:
        for col in required_columns:
            if col not in entry:
                entry[col] = None

    unmatched_df = pd.DataFrame(unmatched)

    # Filtrar jugadores ya emparejados
    try:
        unmatched_df = unmatched_df[~unmatched_df['id_transfermarkt'].isin(matched_players.keys())]
    except KeyError as e:
        logging.error(f"Error al filtrar jugadores no emparejados: {e}")
        return

    # Si existe un archivo previo, combinar y filtrar duplicados
    if os.path.exists(file_path):
        existing_unmatched = pd.read_csv(file_path)
        unmatched_df = pd.concat([existing_unmatched, unmatched_df], ignore_index=True)

    unmatched_df = unmatched_df.drop_duplicates(subset=['id_transfermarkt'], keep='first')

    # Asegurarse de que el archivo final tenga las columnas en el orden esperado
    unmatched_df = unmatched_df[required_columns]

    unmatched_df.to_csv(file_path, index=False)
    logging.info(f"Guardado de no emparejados completado: {file_path}")



def generate_player_dictionary(seasons, rescrape=False):
    """
    Genera el diccionario de jugadores emparejados, con opciones para re-scrapeo.
    """
    all_unmatched = []

    for season in seasons:
        logging.info(f'Procesando temporada: {season}')

        for club_code, club_info in club_mapping.items():
            logging.info(f'Procesando club: {club_info["Club_Name"]} ({season}) (Código: {club_code})')

            # Verificar si la combinación ya fue procesada
            combo_key = (club_code, season)
            if not rescrape and combo_key in scraped_combinations:
                logging.info(f'Combinación {combo_key} ya procesada. Saltando...')
                continue

            # Scraping de Transfermarkt y FBRef
            transfer_players = scrape_transfermarkt(club_code, club_info['Club_Name'], season)
            fbref_players = scrape_fbref(club_info['FBRef_id'], season)

            if not fbref_players:
                logging.warning(f'Información no encontrada para {club_info}. Combinación no registrada.')
                continue

            # Excluir jugadores con match_score=100 ya almacenados
            transfer_players = [
                t_player for t_player in transfer_players
                if t_player['id'] not in all_matches or all_matches[t_player['id']]['match_score'] < 100
            ]

            # Coincidencias y no coincidencias
            matched, unmatched = match_players(transfer_players, fbref_players)

            # Guardar las coincidencias en el diccionario `all_matches`
            for t_player_id, match_info in matched.items():
                existing_match = all_matches.get(t_player_id)
                # Si ya existe, comparar `match_score`
                if existing_match:
                    if match_info['match_score'] > existing_match['match_score']:
                        all_matches[t_player_id] = match_info  # Actualizar con mejor coincidencia
                else:
                    all_matches[t_player_id] = match_info

            # Agregar jugadores no emparejados
            all_unmatched.extend(unmatched)

            # Marcar la combinación como procesada
            scraped_combinations.add(combo_key)

            # Guardado parcial después de cada club
            save_partial_results(all_matches, all_unmatched, scraped_combinations)
            time.sleep(3)  # Pausa para evitar bloqueos

    # Filtrar jugadores no coincidentes, eliminando aquellos ya emparejados
    final_unmatched = [
        player for player in all_unmatched if player['id_transfermarkt'] not in all_matches
    ]

    # Guardado final de resultados
    save_scraped_data(scraped_combinations, all_matches)
    save_unmatched_to_csv(UNMATCHED_PLAYERS_FILE, final_unmatched, all_matches)
    logging.info('Proceso completado exitosamente.')



In [7]:
with open('club_mapping_with_code.json', 'r') as f:
    test_json = json.load(f)
test_json

{'1029': {'Club_Name': 'CA Vélez Sarsfield',
  'FBRef_name': 'Vélez Sarsfield',
  'liga_pais': 'Argentina',
  'FBRef_id': '41c139b6'},
 '1030': {'Club_Name': 'AA Argentinos Juniors',
  'FBRef_name': 'Arg Juniors',
  'liga_pais': 'Argentina',
  'FBRef_id': 'd01a653b'},
 '1070': {'Club_Name': 'CA Colón',
  'FBRef_name': 'Colón',
  'liga_pais': 'Argentina',
  'FBRef_id': '2d84bb17'},
 '1106': {'Club_Name': 'Club de Gimnasia y Esgrima La Plata',
  'FBRef_name': 'Gimnasia–LP',
  'liga_pais': 'Argentina',
  'FBRef_id': '950a95f2'},
 '11831': {'Club_Name': 'CA Tigre',
  'FBRef_name': 'Tigre',
  'liga_pais': 'Argentina',
  'FBRef_id': '0e92bf17'},
 '12301': {'Club_Name': 'CA Aldosivi',
  'FBRef_name': 'Aldosivi',
  'liga_pais': 'Argentina',
  'FBRef_id': 'e5927bd8'},
 '1234': {'Club_Name': 'CA Independiente',
  'FBRef_name': 'Independiente',
  'liga_pais': 'Argentina',
  'FBRef_id': '40bb0ce9'},
 '12454': {'Club_Name': 'CA Sarmiento (Junín)',
  'FBRef_name': 'Sarmiento',
  'liga_pais': 'Argent

In [9]:

# Definir el rango de temporadas y ejecutar la función
seasons = [2024]
generate_player_dictionary(seasons=seasons, rescrape=True)


2024-12-20 16:49:46,286 - INFO - Procesando temporada: 2024
2024-12-20 16:49:46,286 - INFO - Procesando club: CA Vélez Sarsfield (2024) (Código: 1029)
2024-12-20 16:49:53,879 - INFO - Procesando club: AA Argentinos Juniors (2024) (Código: 1030)
2024-12-20 16:50:02,453 - INFO - Procesando club: CA Colón (2024) (Código: 1070)


Error HTTP: 500 Server Error: Internal Server Error for url: https://fbref.com/en/squads/2d84bb17/2024/. No es un 502, 504, ni 429, deteniendo reintentos.
reintentando url con season: 2023-2024


2024-12-20 16:50:10,378 - INFO - Procesando club: Club de Gimnasia y Esgrima La Plata (2024) (Código: 1106)
2024-12-20 16:50:18,982 - INFO - Procesando club: CA Tigre (2024) (Código: 11831)
2024-12-20 16:50:27,485 - INFO - Procesando club: CA Aldosivi (2024) (Código: 12301)


Error HTTP: 500 Server Error: Internal Server Error for url: https://fbref.com/en/squads/e5927bd8/2024/. No es un 502, 504, ni 429, deteniendo reintentos.
reintentando url con season: 2023-2024


2024-12-20 16:50:36,267 - INFO - Procesando club: CA Independiente (2024) (Código: 1234)
2024-12-20 16:50:44,304 - INFO - Procesando club: CA Sarmiento (Junín) (2024) (Código: 12454)
2024-12-20 16:50:52,837 - INFO - Procesando club: CD Godoy Cruz Antonio Tomba (2024) (Código: 12574)
2024-12-20 16:51:01,618 - INFO - Procesando club: CA Newell's Old Boys (2024) (Código: 1286)
2024-12-20 16:51:09,987 - INFO - Procesando club: CA Rosario Central (2024) (Código: 1418)
2024-12-20 16:51:18,407 - INFO - Procesando club: Racing Club (2024) (Código: 1444)
2024-12-20 16:51:26,540 - INFO - Procesando club: Club Atlético Tucumán (2024) (Código: 14554)
2024-12-20 16:51:37,880 - INFO - Procesando club: CA San Lorenzo de Almagro (2024) (Código: 1775)
2024-12-20 16:51:46,754 - INFO - Procesando club: Instituto ACC (2024) (Código: 1829)
2024-12-20 16:51:54,611 - INFO - Procesando club: CA Boca Juniors (2024) (Código: 189)
2024-12-20 16:52:02,923 - INFO - Procesando club: CA Patronato (2024) (Código: 198

...

2024-12-20 17:47:07,692 - INFO - Procesando club: FC St. Pauli (2024) (Código: 35)


Error HTTP: 500 Server Error: Internal Server Error for url: https://fbref.com/en/squads/54864664/2024/. No es un 502, 504, ni 429, deteniendo reintentos.
reintentando url con season: 2023-2024


2024-12-20 17:47:20,012 - INFO - Procesando club: Holstein Kiel (2024) (Código: 269)


Error HTTP: 500 Server Error: Internal Server Error for url: https://fbref.com/en/squads/2ac661d9/2024/. No es un 502, 504, ni 429, deteniendo reintentos.
reintentando url con season: 2023-2024


2024-12-20 17:47:30,795 - INFO - Guardado de no emparejados completado: unmatched_players.csv
2024-12-20 17:47:30,804 - INFO - Proceso completado exitosamente.


In [9]:
import pandas as pd
import json
import os
import requests
from bs4 import BeautifulSoup
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Archivos
PLAYER_MAPPING_FILE = 'player_id_mapping.json'
EXCEL_FILE = 'find_id.xlsx'

# Función para scrapear datos desde FBRef
def scrape_fbref_player(fbref_id):
    url = f"https://fbref.com/en/players/{fbref_id}/-"
    response = make_request(url)
    if response is None:
        logging.warning(f"No se pudo acceder a {url}")
        return None, None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Nombre del jugador
    name_tag = soup.select_one('div.players h1 span')
    nombre_fbref = name_tag.text.strip() if name_tag else None

    # Posición del jugador
    position_tag = soup.select_one('div.players p:contains("Position:")')
    if position_tag:
        # Manejar posición con pie preferido (e.g., "MF (CM-DM) ▪ Footed: Right")
        position_text = position_tag.text.strip()
        posicion_fbref = position_text.replace("Position:", "").strip().split(" ")[0]
    else:
        posicion_fbref = None

    return nombre_fbref, posicion_fbref


In [10]:
import os
import json
import pandas as pd
import logging

def update_json_with_excel(json_file, excel_file, reemplazar=True, DEBUG=False):
    """
    Actualiza un archivo JSON con datos de un archivo Excel, optimizando procesos y reduciendo redundancias.
    
    Args:
        json_file (str): Ruta al archivo JSON.
        excel_file (str): Ruta al archivo Excel.
        reemplazar (bool): Si False, solo actualiza jugadores nuevos en el Excel.
        DEBUG (bool): Si True, imprime información adicional para depuración.
    """
    # Cargar JSON existente
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            json_data = json.load(f)
    else:
        json_data = {"scraped_combinations": [], "players": {}}

    # Cargar DataFrame del Excel
    df_excel = pd.read_excel(excel_file)

    # Filtrar filas inválidas
    df_excel = df_excel[df_excel['fbref_id'].notna()]

    # Extraer jugadores ya presentes en el JSON si reemplazar=False
    if not reemplazar:
        existing_ids = set(json_data.get('players', {}).keys())
        df_excel = df_excel[~df_excel['player_id'].astype(str).isin(existing_ids)]

    # Iterar sobre los datos del Excel
    for _, row in df_excel.iterrows():
        player_id = str(row['player_id'])
        fbref_id = row['fbref_id']
        posicion_transfermarkt = row.get('posicion', None)
        nombre_transfermarkt = row.get('player_name', 'Desconocido')

        if DEBUG:
            logging.info(f'Revisando jugador ID: {player_id}')

        if not player_id or not fbref_id:
            continue

        # Actualizar o añadir datos del jugador
        if player_id in json_data['players']:
            json_data['players'][player_id]['id_fbref'] = fbref_id
            json_data['players'][player_id]['posicion_transfermarkt'] = posicion_transfermarkt
        else:
            json_data['players'][player_id] = {
                'id_transfermarkt': player_id,
                'nombre_transfermarkt': nombre_transfermarkt,
                'posicion_transfermarkt': posicion_transfermarkt,
                'id_fbref': fbref_id,
                'match_score': 100
            }

        # Intentar completar datos desde FBRef
        if not json_data['players'][player_id].get('nombre_fbref') or not json_data['players'][player_id].get('posicion_fbref'):
            nombre_fbref, posicion_fbref = scrape_fbref_player(fbref_id)
            json_data['players'][player_id]['nombre_fbref'] = nombre_fbref
            json_data['players'][player_id]['posicion_fbref'] = posicion_fbref

    # Guardar el JSON actualizado
    with open(json_file, 'w') as f:
        json.dump(json_data, f, indent=4)

    logging.info(f"JSON actualizado con información del archivo {excel_file}")


In [11]:
# Ejecutar la actualización
update_json_with_excel(PLAYER_MAPPING_FILE, EXCEL_FILE, reemplazar=False, DEBUG=True)

2024-12-09 13:04:54,235 - INFO - JSON actualizado con información del archivo find_id.xlsx


In [12]:
df_excel = pd.read_excel(EXCEL_FILE)

df_filtered = df_excel[df_excel['fbref_id'].notna()]
df_filtered

,player_name,player_url,player_id,fbref_id,age,nationality,comunitario,extranjero,position,transfer_fee,...,Internacional_long_season_ant2,Local_long_season_ant2,Club_playing_ant2,id_club_playing_ant2,competition_season_tuples_ant2,Internacional_Goals Conceded_ant2,Internacional_Clean Sheets_ant2,Local_Goals Conceded_ant2,Local_Clean Sheets_ant2,season_stats
0,Abel Luciatti,/abel-luciatti/profil/spieler/220751,220751,ba0f7e5f,30,['Argentina'],0,0,Defensa central,0,...,0.0,0.0,['CA Tigre'],['11831'],"{'Local': [('Prim. Nacional - Playoff', '2021'...",NaN,NaN,NaN,NaN,2022
1,Wilder Cartagena,/wilder-cartagena/profil/spieler/224669,224669,9f0f5d15,26,['Perú'],0,1,Pivote,0,...,0.0,0.0,['Club Alianza Lima'],['184'],"{'Internacional': [('Libertadores', '2019')], ...",NaN,NaN,NaN,NaN,2020
2,Matías García,/matias-garcia/profil/spieler/267393,267393,91e7fc9a,31,"['Argentina', 'Italia']",1,0,Extremo izquierdo,0,...,0.0,0.0,"['Club de Gimnasia y Esgrima La Plata', 'FC Ju...","['1106', '49283']","{'Local': [('Copa de la Liga - Playoffs', '202...",NaN,NaN,NaN,NaN,2021
3,Javier Báez,/javier-baez/profil/spieler/87360,87360,56d59ae0,31,"['Paraguay', 'Argentina']",0,0,Defensa central,0,...,0.0,0.0,['Club Guaraní'],['11855'],"{'Local': [('Primera División Clausura', '2019...",NaN,NaN,NaN,NaN,2020
4,Alex Ibacache,/alex-ibacache/profil/spieler/571083,571083,68a38211,24,['Chile'],0,1,Lateral izquierdo,0,...,0.0,0.0,['CDP Curicó Unido'],['19027'],"{'Local': [('Primera División', '2020')]}",NaN,NaN,NaN,NaN,2021
5,Emanuel Hernández,/emanuel-hernandez/profil/spieler/651854,651854,0fab7750,25,['Uruguay'],0,1,Defensa central,0,...,0.0,0.0,['Jaguares de Córdoba'],['39252'],"{'Local': [('Liga Dimayor I', '2020'), ('Copa ...",NaN,NaN,NaN,NaN,2021
6,Lázaro Romero,/lazaro-romero/profil/spieler/971557,971557,9c4136a3,28,['Argentina'],0,0,Delantero centro,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022
7,Leonardo Ramos,/leonardo-ramos/profil/spieler/341298,341298,2271da97,32,['Argentina'],0,0,Delantero centro,0,...,0.0,0.0,['Club León FC'],['4941'],"{'Internacional': [('CONCACAF Champions Cup', ...",NaN,NaN,NaN,NaN,2020
8,Carlos Protesoni,/carlos-protesoni/profil/spieler/431190,431190,9fb45456,24,"['Uruguay', 'Italia']",1,1,Pivote,0,...,0.0,0.0,['CA Independiente'],['1234'],"{'Internacional': [('Sudamericana', '2020')], ...",NaN,NaN,NaN,NaN,2021
9,Yair Arismendi,/yair-arismendi/profil/spieler/440006,440006,9438188a,24,['Argentina'],0,0,Extremo izquierdo,0,...,0.0,0.0,['AA Estudiantes (Río Cuarto)'],['14602'],"{'Local': [('Transición Primera Nacional', '20...",NaN,NaN,NaN,NaN,2021


In [10]:
with open('player_id_mapping.json', 'r') as f:
    player_id_mapping = json.load(f)
player_ids = player_id_mapping['players']
print(len(player_ids))  # 12413 

12495


In [11]:

player_club_mapping = player_id_mapping['scraped_combinations']
print(len(player_club_mapping))

1768


In [12]:
# Eliminacion de unmmatched de las claves agregadas

# Obtener las claves de jugadores ya presentes en el JSON
player_ids = set(player_id_mapping['players'].keys())  # Claves del JSON como conjunto

# Filtrar el DataFrame, manteniendo solo las filas que no están en el JSON
filtered_unmatched = unmatched[~unmatched['id_transfermarkt'].astype(str).isin(player_ids)]

# Sobrescribir el archivo CSV con las filas filtradas
filtered_unmatched.to_csv(UNMATCHED_PLAYERS_FILE, index=False)

print(f"Filtrado completado. Se eliminaron {len(unmatched) - len(filtered_unmatched)} filas.")

NameError: name 'unmatched' is not defined